# Trying to connect 1 FED to RbPi4B 

Read the manuals on this  [link](https://www.raspberrypi.com/documentation/computers/getting-started.html) to setup your Raspberry Pi board, once the operating system is installed on your board, connect mouse and keyboard to the board (USB ports), get Wifi connected, and follow the steps below to prepare the board to run the python script.

### open the Terminal on your RbPi and run  the commands below one by one,


    1) sudo apt-get update
    2) sudo apt-get upgrade
    3) sudo apt-get install python3 python3-pip

### Now let's create a new virtual environment on our RbPi:

    1) sudo apt-get install python3-venv

    2) python3 -m venv HOMEPHOTOFED

    3) source HOMEPHOTOFED/bin/activate

### Now install pyserial to enable serial communication with FED3

    pip install pyserial
    
    sudo apt-get install python3-rpi.gpio

    pip install RPi.GPIO

    pip3 install jupyter
    
### Ensure the serial port is enabled
    sudo raspi-config
Go to Interfacing Options.
Select Serial.
Disable the login shell over the serial connection.
Enable the serial hardware.

### Reboot the board after making those changes 
    sudo reboot (Or just by closing the interface window it might ask you to reboot)

### Optional if you want to monitor serial communication from FED3 and test the data being sent FED3
    1) sudo apt-get install screen
    2) screen /dev/ttyUSB0 115200 (But to find the port name of your FED3, you need to follow the substep below)
        2-1) Connect the FED3 to the USB port on your RbPi and switch on the FED3
        2-2) On your RbPi terminal run this command " lsusb " to see whether Adafruit Feather board is connected
        2-3) run sudo dmesg -C ( it clear the current dmesg buffer)
        2-4) reboot your FED3
        2-5) run dmesg | grep tty
        2-6) you will see somthing like [ 1234.56789] cdc_acm 1-1:1.0: ttyACM0: USB ACM device  # the ttyACM0 is your FED3 port name

    3) now run screen/dev/ttyACM0 115200
    When you run this a new window will pop up and you should see data coming from FED3 by interacting with it

    4) to exit the screen press Ctrl+A and then K and followed by Y to confirm
    
        
### Open your jupyter notebook and setup your notebook
    run jupyter notebook --no-browser --port=8888
    create directories for your notebook and make a jupyter notebook file 
    You can transfer the codes via USB or just get them online from your Github profile

### Now you should be ready to run the code below and test FED-PI-TDT

    

    
    



# <font color = "red">Code beow sends signals via 3 pins on the board -- using same for pellet,pelletinwell, right poke and Rightwithpellet, etc <font/>

In [1]:
import RPi.GPIO as GPIO
import serial
import threading
import time

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins (matching your previous Arduino pin setup)
gpio_pins = {
    "LeftPoke": 9,          # Pin for Left poke event
    "RightPoke": 11,        # Pin for Right poke event
    "Pellet": 13,           # Pin for PelletInWell and PelletTaken
}

# Set all pins as output
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)  # Ensure all pins start LOW

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event():
    global pellet_in_well
    if pellet_in_well:
        # If a pellet was in the well, signal pellet taken
        GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
        time.sleep(0.01)
        pellet_in_well = False
        print("Pellet taken, signal turned OFF.")
    else:
        # Pellet is dispensed, keep the signal high
        GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
        pellet_in_well = True
        print("Pellet dispensed in well, signal ON.")

# Function to process each event
def process_event(event_type):
    print(f"Processing event: {event_type}")
    
    if event_type == "LeftPoke":
        send_ttl_signal(gpio_pins["LeftPoke"])
        print("Left poke event triggered.")
        
    elif event_type == "RightPoke":
        send_ttl_signal(gpio_pins["RightPoke"])
        print("Right poke event triggered.")
        
    elif event_type == "LeftWithPellet":
        GPIO.output(gpio_pins["LeftPoke"], GPIO.HIGH)
        print("Left poke with pellet, signal HIGH.")
        
    elif event_type == "RightWithPellet":
        GPIO.output(gpio_pins["RightPoke"], GPIO.HIGH)
        print("Right poke with pellet, signal HIGH.")
        
    elif event_type == "Pellet":
        handle_pellet_event()

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line:
            event_type = line.split(",")[8]  # Extract the event type (assuming it's at index 8)
            print(f"Received event: {event_type}")
            process_event(event_type)

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyUSB0": "FED1",  # Add more FED ports if needed
    # "/dev/ttyUSB1": "FED2",  # Example of additional FEDs
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    threading.Thread(target=read_from_fed, args=(port,)).start()

# Cleanup GPIO when script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    print("Cleaning up GPIO and exiting.")
    GPIO.cleanup()


ModuleNotFoundError: No module named 'RPi'

# <font color = "red">Code beow sends TTL signals via different pins for each event, however Pellet and PelletInWell are using the same pin  <font/>

In [ ]:
import RPi.GPIO as GPIO
import serial
import threading
import time

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins (using separate pins for each event, except PelletInWell and PelletTaken)
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17, Physical Pin 11)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27, Physical Pin 13)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22, Physical Pin 15)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23, Physical Pin 16)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24, Physical Pin 18)
}

# Set all pins as output
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)  # Ensure all pins start LOW

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event():
    global pellet_in_well
    if pellet_in_well:
        # If a pellet was in the well, signal pellet taken
        GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
        time.sleep(0.01)
        pellet_in_well = False
        print("Pellet taken, signal turned OFF.")
    else:
        # Pellet is dispensed, keep the signal high
        GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
        pellet_in_well = True
        print("Pellet dispensed in well, signal ON.")

# Function to process each event
def process_event(event_type):
    print(f"Processing event: {event_type}")
    
    if event_type == "LeftPoke":
        send_ttl_signal(gpio_pins["LeftPoke"])
        print("Left poke event triggered.")
        
    elif event_type == "RightPoke":
        send_ttl_signal(gpio_pins["RightPoke"])
        print("Right poke event triggered.")
        
    elif event_type == "LeftWithPellet":
        GPIO.output(gpio_pins["LeftWithPellet"], GPIO.HIGH)
        print("Left poke with pellet, signal HIGH.")
        
    elif event_type == "RightWithPellet":
        GPIO.output(gpio_pins["RightWithPellet"], GPIO.HIGH)
        print("Right poke with pellet, signal HIGH.")
        
    elif event_type == "Pellet":
        handle_pellet_event()

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line:
            event_type = line.split(",")[8]  # Extract the event type (assuming it's at index 8)
            print(f"Received event: {event_type}")
            process_event(event_type)

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Add more FED ports if needed
    # "/dev/ttyACM1": "FED2",  # Example of additional FEDs
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    threading.Thread(target=read_from_fed, args=(port,)).start()

# Cleanup GPIO when script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    print("Cleaning up GPIO and exiting.")
    GPIO.cleanup()


# LED TEST

In [ ]:
import RPi.GPIO as GPIO
import time

# Set up the GPIO pin for the LED
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(17, GPIO.OUT)  # Using GPIO 17 (Pin 11)

# Test LED by blinking it
try:
    while True:
        GPIO.output(17, GPIO.HIGH)  # Turn on the LED
        print("LED ON")
        time.sleep(1)  # Keep it on for 1 second
        GPIO.output(17, GPIO.LOW)   # Turn off the LED
        print("LED OFF")
        time.sleep(1)  # Keep it off for 1 second
except KeyboardInterrupt:
    print("Exiting and cleaning up GPIO")
finally:
    GPIO.cleanup()  # Clean up GPIO settings


In [ ]:
import RPi.GPIO as GPIO
import time

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for the LEDs (same as your setup)
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17, Physical Pin 11)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27, Physical Pin 13)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22, Physical Pin 15)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23, Physical Pin 16)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24, Physical Pin 18)
}

# Set all pins as output and initially set them to LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)

# Test each LED
try:
    while True:
        print("Testing LeftPoke LED...")
        GPIO.output(gpio_pins["LeftPoke"], GPIO.HIGH)  # Turn on LeftPoke LED
        time.sleep(1)
        GPIO.output(gpio_pins["LeftPoke"], GPIO.LOW)  # Turn off LeftPoke LED

        print("Testing RightPoke LED...")
        GPIO.output(gpio_pins["RightPoke"], GPIO.HIGH)  # Turn on RightPoke LED
        time.sleep(1)
        GPIO.output(gpio_pins["RightPoke"], GPIO.LOW)  # Turn off RightPoke LED

        print("Testing LeftWithPellet LED...")
        GPIO.output(gpio_pins["LeftWithPellet"], GPIO.HIGH)  # Turn on LeftWithPellet LED
        time.sleep(1)
        GPIO.output(gpio_pins["LeftWithPellet"], GPIO.LOW)  # Turn off LeftWithPellet LED

        print("Testing RightWithPellet LED...")
        GPIO.output(gpio_pins["RightWithPellet"], GPIO.HIGH)  # Turn on RightWithPellet LED
        time.sleep(1)
        GPIO.output(gpio_pins["RightWithPellet"], GPIO.LOW)  # Turn off RightWithPellet LED

        print("Testing Pellet LED...")
        GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Turn on Pellet LED
        time.sleep(1)
        GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Turn off Pellet LED

except KeyboardInterrupt:
    print("Exiting and cleaning up GPIO")
    GPIO.cleanup()


# The LED sctips works fine, but FED does not sends TTLs to RbPi, now I am adjusting the code to make sure FED3 is communicating with the board

In [ ]:
import RPi.GPIO as GPIO
import serial
import threading
import time

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for the LEDs (same as your setup)
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17, Physical Pin 11)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27, Physical Pin 13)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22, Physical Pin 15)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23, Physical Pin 16)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24, Physical Pin 18)
}

# Set all pins as output and initially set them to LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event():
    global pellet_in_well
    if pellet_in_well:
        # If a pellet was in the well, signal pellet taken
        GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
        time.sleep(0.01)
        pellet_in_well = False
        print("Pellet taken, signal turned OFF.")
    else:
        # Pellet is dispensed, keep the signal high
        GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
        pellet_in_well = True
        print("Pellet dispensed in well, signal ON.")

# Function to process each event
def process_event(event_type):
    print(f"Processing event: {event_type}")
    
    if event_type == "LeftPoke":
        send_ttl_signal(gpio_pins["LeftPoke"])
        print("Left poke event triggered.")
        
    elif event_type == "RightPoke":
        send_ttl_signal(gpio_pins["RightPoke"])
        print("Right poke event triggered.")
        
    elif event_type == "LeftWithPellet":
        GPIO.output(gpio_pins["LeftWithPellet"], GPIO.HIGH)
        print("Left poke with pellet, signal HIGH.")
        
    elif event_type == "RightWithPellet":
        GPIO.output(gpio_pins["RightWithPellet"], GPIO.HIGH)
        print("Right poke with pellet, signal HIGH.")
        
    elif event_type == "Pellet":
        handle_pellet_event()

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()
        print(f"Raw FED3 data: {line}")  # Debug print to show raw FED3 data
        if line:
            # Adjust this based on your specific FED3 format
            try:
                event_type = line.split(",")[8]  # Extract the event type (assuming it's at index 8)
                print(f"Extracted event type: {event_type}")  # Debug print to confirm event extraction
                process_event(event_type)
            except IndexError:
                print("Error: Unable to extract event type, check data format.")

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Add more FED ports if needed
    # "/dev/ttyACM1": "FED2",  # Example of additional FEDs
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    threading.Thread(target=read_from_fed, args=(port,)).start()

# Cleanup GPIO when script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    print("Cleaning up GPIO and exiting.")
    GPIO.cleanup()


In [ ]:
import RPi.GPIO as GPIO
import serial
import threading
import time

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for the LEDs
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17, Physical Pin 11)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27, Physical Pin 13)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22, Physical Pin 15)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23, Physical Pin 16)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24, Physical Pin 18)
}

# Set all pins as output and initially set them to LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    print(f"Sending TTL signal to pin {pin}")  # Debugging print
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)
    print(f"TTL signal turned OFF for pin {pin}")  # Debugging print

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event():
    global pellet_in_well
    if pellet_in_well:
        # If a pellet was in the well, signal pellet taken
        GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
        time.sleep(0.01)
        pellet_in_well = False
        print("Pellet taken, signal turned OFF.")
    else:
        # Pellet is dispensed, keep the signal high
        GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
        pellet_in_well = True
        print("Pellet dispensed in well, signal ON.")

# Function to process each event
def process_event(event_type):
    print(f"Processing event: {event_type}")
    
    if event_type == "LeftPoke":
        send_ttl_signal(gpio_pins["LeftPoke"])
        print("Left poke event triggered.")
        
    elif event_type == "RightPoke":
        send_ttl_signal(gpio_pins["RightPoke"])
        print("Right poke event triggered.")
        
    elif event_type == "LeftWithPellet":
        print(f"Setting GPIO {gpio_pins['LeftWithPellet']} HIGH for LeftWithPellet event")
        GPIO.output(gpio_pins["LeftWithPellet"], GPIO.HIGH)
        
    elif event_type == "RightWithPellet":
        print(f"Setting GPIO {gpio_pins['RightWithPellet']} HIGH for RightWithPellet event")
        GPIO.output(gpio_pins["RightWithPellet"], GPIO.HIGH)
        
    elif event_type == "Pellet":
        handle_pellet_event()

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()
        print(f"Raw FED3 data: {line}")  # Debug print to show raw FED3 data
        if line:
            try:
                event_type = line.split(",")[8]  # Extract the event type (assuming it's at index 8)
                print(f"Extracted event type: {event_type}")  # Debug print to confirm event extraction
                process_event(event_type)
            except IndexError:
                print("Error: Unable to extract event type, check data format.")

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Add more FED ports if needed
    # "/dev/ttyACM1": "FED2",  # Example of additional FEDs
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    threading.Thread(target=read_from_fed, args=(port,)).start()

# Cleanup GPIO when script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    print("Cleaning up GPIO and exiting.")
    GPIO.cleanup()


In [ ]:
import serial
import time

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()
        print(f"Raw FED3 data: {line}")  # Debug print to show raw FED3 data
        time.sleep(1)  # Add a small delay to avoid overwhelming the output

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Add more FED ports if needed
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    read_from_fed(port)


In [ ]:
import serial
import time

# Function to filter and read from serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()

        # Only process the data if it's not empty and contains meaningful data
        if line and len(line.split(",")) > 8:
            print(f"Raw FED3 data: {line}")  # Only print relevant data
            
            # For debugging: check if the interaction contains specific event types
            try:
                event_type = line.split(",")[8]  # Extract event type (at index 8)
                print(f"Extracted event type: {event_type}")
            except IndexError:
                print("Error: Unable to extract event type, check data format.")
        else:
            # Ignore noise or irrelevant data
            print("No relevant event data received, skipping...")
        time.sleep(1)  # Avoid overloading the output

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Adjust the port if necessary
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    read_from_fed(port)


In [ ]:
import RPi.GPIO as GPIO
import serial
import threading
import datetime

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for the TTL signals
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17, Physical Pin 11)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27, Physical Pin 13)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22, Physical Pin 15)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23, Physical Pin 16)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24, Physical Pin 18)
}

# Set all pins as output and initially set them to LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)  # Ensure all pins start LOW

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Function to send TTL pulse for poke events
def send_ttl_signal(pin):
    print(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)
    print(f"TTL signal sent to pin {pin}")

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event():
    global pellet_in_well
    if pellet_in_well:
        # If a pellet was in the well, signal pellet taken
        GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
        pellet_in_well = False
        print("Pellet taken, signal turned OFF.")
    else:
        # Pellet is dispensed, keep the signal high
        GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
        pellet_in_well = True
        print("Pellet dispensed in well, signal ON.")

# Function to process each event
def process_event(event_type):
    print(f"Processing event: {event_type}")
    
    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
        print("Left poke event triggered.")
        
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
        print("Right poke event triggered.")
        
    elif event_type == "LeftWithPellet":
        print(f"Setting GPIO {gpio_pins['LeftWithPellet']} HIGH for LeftWithPellet event")
        GPIO.output(gpio_pins["LeftWithPellet"], GPIO.HIGH)
        
    elif event_type == "RightWithPellet":
        print(f"Setting GPIO {gpio_pins['RightWithPellet']} HIGH for RightWithPellet event")
        GPIO.output(gpio_pins["RightWithPellet"], GPIO.HIGH)
        
    elif event_type == "Pellet":
        handle_pellet_event()

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()
        print(f"Raw FED3 data: {line}")  # Debug print to show raw FED3 data
        if line:
            try:
                event_type = line.split(",")[8]  # Extract event type (assuming it's at index 8)
                print(f"Extracted event type: {event_type}")
                process_event(event_type)
            except IndexError:
                print("Error: Unable to extract event type, check data format.")

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Adjust the port if necessary
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    threading.Thread(target=read_from_fed, args=(port,)).start()

# Cleanup GPIO when script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    print("Cleaning up GPIO and exiting.")
    GPIO.cleanup()


In [ ]:
import RPi.GPIO as GPIO
import serial
import threading
import time

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins (same as your setup)
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17, Physical Pin 11)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27, Physical Pin 13)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22, Physical Pin 15)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23, Physical Pin 16)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24, Physical Pin 18)
}

# Set all pins as output and initially set them to LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)  # Ensure all pins start LOW

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Function to send TTL pulse for poke events
def send_ttl_signal(pin):
    print(f"Sending TTL signal to pin {pin}")  # Debug: Show which pin is triggered
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)
    print(f"TTL signal sent to pin {pin}")

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event():
    global pellet_in_well
    if pellet_in_well:
        # If a pellet was in the well, signal pellet taken
        GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
        pellet_in_well = False
        print("Pellet taken, signal turned OFF.")
    else:
        # Pellet is dispensed, keep the signal high
        GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
        pellet_in_well = True
        print("Pellet dispensed in well, signal ON.")

# Function to process each event
def process_event(event_type):
    print(f"Processing event: {event_type}")  # Debug: Show event type received
    
    if event_type == "Left":
        print("Left poke detected")
        send_ttl_signal(gpio_pins["LeftPoke"])
        
    elif event_type == "Right":
        print("Right poke detected")
        send_ttl_signal(gpio_pins["RightPoke"])
        
    elif event_type == "LeftWithPellet":
        print(f"Left with pellet detected, setting GPIO {gpio_pins['LeftWithPellet']} HIGH")
        GPIO.output(gpio_pins["LeftWithPellet"], GPIO.HIGH)
        
    elif event_type == "RightWithPellet":
        print(f"Right with pellet detected, setting GPIO {gpio_pins['RightWithPellet']} HIGH")
        GPIO.output(gpio_pins["RightWithPellet"], GPIO.HIGH)
        
    elif event_type == "Pellet":
        handle_pellet_event()

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()
        print(f"Raw FED3 data: {line}")  # Debug: Print raw FED3 data to confirm
        if line:
            try:
                event_type = line.split(",")[8]  # Extract event type (assuming it's at index 8)
                print(f"Extracted event type: {event_type}")  # Debug: Show extracted event type
                process_event(event_type)
            except IndexError:
                print("Error: Unable to extract event type, check data format.")

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Adjust the port if necessary
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    threading.Thread(target=read_from_fed, args=(port,)).start()

# Cleanup GPIO when script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    print("Cleaning up GPIO and exiting.")
    GPIO.cleanup()


#################################################

In [ ]:
import RPi.GPIO as GPIO
import serial
import threading
import time

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins (matching the Arduino setup for LED control)
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17, Physical Pin 11)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27, Physical Pin 13)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22, Physical Pin 15)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23, Physical Pin 16)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24, Physical Pin 18)
}

# Set all pins as output and ensure they start LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)  # Ensure all pins start LOW

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Function to send TTL pulse for poke events (LEDs blink)
def send_ttl_signal(pin):
    print(f"Sending TTL signal to pin {pin}")  # Debug: Show which pin is triggered
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)
    print(f"TTL signal sent to pin {pin}")

# Function to handle PelletInWell/PelletTaken logic
def handle_pellet_event():
    global pellet_in_well
    if pellet_in_well:
        # If a pellet was in the well, signal pellet taken
        GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
        pellet_in_well = False
        print("Pellet taken, signal turned OFF.")
    else:
        # Pellet is dispensed, keep the signal high
        GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
        pellet_in_well = True
        print("Pellet dispensed in well, signal ON.")

# Function to process each event and send corresponding TTL signals
def process_event(event_type):
    print(f"Processing event: {event_type}")  # Debug: Show event type received
    
    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
        print("Left poke event triggered.")
        
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
        print("Right poke event triggered.")
        
    elif event_type == "LeftWithPellet":
        GPIO.output(gpio_pins["LeftWithPellet"], GPIO.HIGH)
        print("Left with pellet event, signal HIGH.")
        
    elif event_type == "RightWithPellet":
        GPIO.output(gpio_pins["RightWithPellet"], GPIO.HIGH)
        print("Right with pellet event, signal HIGH.")
        
    elif event_type == "Pellet":
        handle_pellet_event()

# Function to read from the serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()
        print(f"Raw FED3 data: {line}")  # Debug: Print raw FED3 data
        if line:
            try:
                event_type = line.split(",")[8]  # Extract the event type (assuming it's at index 8)
                print(f"Extracted event type: {event_type}")  # Debug: Show extracted event type
                process_event(event_type)
            except IndexError:
                print("Error: Unable to extract event type, check data format.")

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Adjust the port if necessary
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    threading.Thread(target=read_from_fed, args=(port,)).start()

# Cleanup GPIO when the script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    print("Cleaning up GPIO and exiting.")
    GPIO.cleanup()


$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

In [ ]:
import RPi.GPIO as GPIO
import serial
import threading
import time

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins (new pin numbers as requested)
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17, Physical Pin 11)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27, Physical Pin 13)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22, Physical Pin 15)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23, Physical Pin 16)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24, Physical Pin 18)
}

# Set all pins as output and ensure they start LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)  # Ensure all pins start LOW

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    print(f"Sending TTL signal to pin {pin}")  # Debug: Show which pin is triggered
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)
    print(f"TTL signal sent to pin {pin}")

# Function to handle PelletInWell/PelletTaken logic
def handle_pellet_event():
    global pellet_in_well
    if pellet_in_well:
        # If a pellet was in the well, signal pellet taken
        GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
        time.sleep(0.01)
        pellet_in_well = False
        print("Pellet taken, signal turned OFF.")
    else:
        # Pellet is dispensed, keep the signal high
        GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
        pellet_in_well = True
        print("Pellet dispensed in well, signal ON.")

# Function to process each event and send corresponding TTL signals
def process_event(event_type):
    print(f"Processing event: {event_type}")  # Debug: Show event type received
    
    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
        print("Left poke event triggered.")
        
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
        print("Right poke event triggered.")
        
    elif event_type == "LeftWithPellet":
        GPIO.output(gpio_pins["LeftWithPellet"], GPIO.HIGH)
        print("Left with pellet event, signal HIGH.")
        
    elif event_type == "RightWithPellet":
        GPIO.output(gpio_pins["RightWithPellet"], GPIO.HIGH)
        print("Right with pellet event, signal HIGH.")
        
    elif event_type == "Pellet":
        handle_pellet_event()

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line:
            try:
                event_type = line.split(",")[8]  # Extract the event type (assuming it's at index 8)
                print(f"Received event: {event_type}")  # Debug: Show extracted event type
                process_event(event_type)
            except IndexError:
                print("Error: Unable to extract event type, check data format.")

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Adjust the port if necessary
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    threading.Thread(target=read_from_fed, args=(port,)).start()

# Cleanup GPIO when the script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    print("Cleaning up GPIO and exiting.")
    GPIO.cleanup()


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins (matching your previous Arduino pin setup)
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event
    "RightPoke": 27,         # Pin for Right poke event
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event
    "RightWithPellet": 23,   # Pin for Right poke with pellet event
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken
}

# Set all pins as output
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)  # Ensure all pins start LOW

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    print(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event():
    global pellet_in_well
    if pellet_in_well:
        GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
        time.sleep(0.01)
        pellet_in_well = False
        print("Pellet taken, signal turned OFF.")
    else:
        GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
        pellet_in_well = True
        print("Pellet dispensed in well, signal ON.")

# Function to process each event
def process_event(event_type):
    print(f"Processing event: {event_type}")
    
    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
        print("Left poke event triggered.")
        
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
        print("Right poke event triggered.")
        
    elif event_type == "LeftWithPellet":
        GPIO.output(gpio_pins["LeftWithPellet"], GPIO.HIGH)
        print("Left poke with pellet, signal HIGH.")
        
    elif event_type == "RightWithPellet":
        GPIO.output(gpio_pins["RightWithPellet"], GPIO.HIGH)
        print("Right poke with pellet, signal HIGH.")
        
    elif event_type == "Pellet":
        handle_pellet_event()

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line:
            data_list = line.split(",")  # Split the data string into a list
            print(f"Raw FED3 data: {data_list}")  # Debug: Show the full data received
            if len(data_list) == len(column_headers):  # Ensure the data matches the column length
                event_type = data_list[8]  # "Event" field contains event type
                print(f"Extracted event type: {event_type}")
                process_event(event_type)
            else:
                print("Warning: Data length does not match header length, skipping.")

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Add more FED ports if needed
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    threading.Thread(target=read_from_fed, args=(port,)).start()

# Cleanup GPIO when script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    print("Cleaning up GPIO and exiting.")
    GPIO.cleanup()


********************************************************

In [ ]:
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for the TTL output (same as your original setup)
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17, Physical Pin 11)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27, Physical Pin 13)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22, Physical Pin 15)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23, Physical Pin 16)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24, Physical Pin 18)
}

# Set all pins as output and initially set them to LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    print(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event():
    global pellet_in_well
    if pellet_in_well:
        GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
        time.sleep(0.01)
        pellet_in_well = False
        print("Pellet taken, signal turned OFF.")
    else:
        GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
        pellet_in_well = True
        print("Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type):
    print(f"Processing event: {event_type}")
    
    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])  # Trigger Left Poke signal
        print("Left poke event triggered.")
        
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])  # Trigger Right Poke signal
        print("Right poke event triggered.")
        
    elif event_type == "LeftWithPellet":
        GPIO.output(gpio_pins["LeftWithPellet"], GPIO.HIGH)  # Keep LeftWithPellet signal HIGH
        print("Left poke with pellet, signal HIGH.")
        
    elif event_type == "RightWithPellet":
        GPIO.output(gpio_pins["RightWithPellet"], GPIO.HIGH)  # Keep RightWithPellet signal HIGH
        print("Right poke with pellet, signal HIGH.")
        
    elif event_type == "Pellet":
        handle_pellet_event()  # Handle PelletInWell and PelletTaken

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line:
            data_list = line.split(",")  # Split the data string into a list
            print(f"Raw FED3 data: {data_list}")  # Debug: Show the full data received
            if len(data_list) == len(column_headers):  # Ensure the data matches the column length
                event_type = data_list[9]  # "Event" field contains event type
                print(f"Extracted event type: {event_type}")
                process_event(event_type)  # Send TTL signal for the event
            else:
                print("Warning: Data length does not match header length, skipping.")

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Add more FED ports if needed
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    threading.Thread(target=read_from_fed, args=(port,)).start()

# Cleanup GPIO when script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    print("Cleaning up GPIO and exiting.")
    GPIO.cleanup()


# Latest adjustment

In [ ]:
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for the TTL output (same as your original setup)
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17, Physical Pin 11)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27, Physical Pin 13)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22, Physical Pin 15)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23, Physical Pin 16)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24, Physical Pin 18)
}

# Set all pins as output and initially set them to LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    print(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type):
    global pellet_in_well
    if event_type == "Pellet":
        if pellet_in_well:
            GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
            print("Pellet taken, signal turned OFF.")
            pellet_in_well = False  # Update state
            send_ttl_signal(gpio_pins["Pellet"])  # Send a short TTL pulse for PelletTaken
        else:
            print("No pellet was in the well, no signal for pellet taken.")
    elif event_type == "PelletInWell":
        GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
        pellet_in_well = True
        print("Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type, timestamp):
    print(f"[{timestamp}] Processing event: {event_type}")
    
    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])  # Trigger Left Poke signal
        print("Left poke event triggered.")
        
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])  # Trigger Right Poke signal
        print("Right poke event triggered.")
        
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftWithPellet"])  # Trigger LeftWithPellet signal briefly
        print("Left poke with pellet, signal triggered.")
        
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightWithPellet"])  # Trigger RightWithPellet signal briefly
        print("Right poke with pellet, signal triggered.")
        
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type)  # Handle PelletInWell and PelletTaken

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line:
            data_list = line.split(",")  # Split the data string into a list
            print(f"Raw FED3 data: {data_list}")  # Debug: Show the full data received
            if len(data_list) == len(column_headers):  # Ensure the data matches the column length
                event_type = data_list[9]  # "Event" field contains event type
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp
                print(f"Extracted event type: {event_type} at {timestamp}")
                process_event(event_type, timestamp)  # Send TTL signal for the event
            else:
                print("Warning: Data length does not match header length, skipping.")

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Add more FED ports if needed
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    threading.Thread(target=read_from_fed, args=(port,)).start()

# Cleanup GPIO when script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    print("Cleaning up GPIO and exiting.")
    GPIO.cleanup()
